# QED-to-Lattice Closure Loop — Phase II Deliverable (2025-10-28T03:50:08)

Objective: One deterministic pipeline (one normalization, no knob-turning) that:
1. Takes **Bridge** outputs { ω_c, m_Γ, χ }.
2. Computes stiffnesses **K₁, K₂, K₃** (U(1), SU(2), SU(3)).
3. Runs 1‑loop (and optionally 2‑loop) RGEs down to **M_Z** to get **α(M_Z)**, **sin²θ_W(M_Z)**, **α_s(M_Z)**.
4. Predicts **lattice spacing** `a` and **string tension** `σ` via **MATH‑YM‑003** mapping.
5. Compares to **PDG** (couplings) and **FLAG/MILC** (a, σ), without retuning.

> This notebook is a scaffold with explicit computational steps. You can lock the mapping (one normalization) and press **Run All**.

In [1]:
# Imports
import numpy as np
import pandas as pd
from math import log, pi, sqrt

# Constants (can be adjusted if you want a different input scale or reference values)
MZ_GeV = 91.1876  # Z boson mass (GeV)

# PDG-like targets at MZ (reference values for comparison)
PDG_alpha_em_inv = 127.955  # alpha_em^-1(MZ)
PDG_sin2_thetaW = 0.23122   # sin^2 theta_W(MZ)
PDG_alpha_s = 0.1179        # alpha_s(MZ)

targets = {
    "alpha_em_inv_MZ": PDG_alpha_em_inv,
    "sin2_thetaW_MZ": PDG_sin2_thetaW,
    "alpha_s_MZ": PDG_alpha_s,
}

targets

{'alpha_em_inv_MZ': 127.955, 'sin2_thetaW_MZ': 0.23122, 'alpha_s_MZ': 0.1179}

## Step 0 — Inputs from the Bridge

> Provide **ω_c [Hz]**, **m_Γ [MeV]**, **χ [dimensionless susceptibility]**.
> Choose a **Bridge scale** Λ_B (GeV) at which the stiffness dictionary is defined.

In [2]:
# --- USER INPUTS (Bridge) ---
omega_c = 1.0e23    # Hz
m_Gamma_MeV = 17.0  # MeV
chi = 0.85          # unitless

# Bridge normalization scale (GeV) — where K_i are defined via the dictionary
Lambda_B_GeV = 1000.0  # 1 TeV default; adjust if your Bridge prefers a different Λ_B

omega_c, m_Gamma_MeV, chi, Lambda_B_GeV

(1e+23, 17.0, 0.85, 1000.0)

## Step 1 — Stiffness dictionary (MATH‑QED‑001…005, MATH‑YM‑001…003)

**One normalization, no knob-turning**: define a single scale `Λ_0` and fixed, explicit forms that map `{ω_c, m_Γ, χ}` → `{K₁, K₂, K₃}`.

> **Example (placeholder)**:  
> We treat the stiffnesses as
> \[
> K_1 = \Lambda_0 \left(rac{\omega_c}{\omega_0}ight)^{1/3}\chi^{\,p_1},\quad
> K_2 = \Lambda_0 \left(rac{m_\Gamma}{m_0}ight)^{1/2}\chi^{\,p_2},\quad
> K_3 = \Lambda_0 \left(rac{\omega_c m_\Gamma}{\omega_0 m_0}ight)^{1/4}\chi^{\,p_3}.
> \]
> with a **single** normalization Λ₀ and **fixed exponents** p₁,p₂,p₃ chosen from theory (no fitting here).
> 
> Then define **gauge couplings at Λ_B** by a proportional map \( lpha_i(\Lambda_B) \propto 1/K_i^2 \) up to the **single** normalization.
> 
> Replace the placeholders below with your final forms *once fixed in the paper text*.

In [3]:
# --- SINGLE NORMALIZATION AND FIXED FORMS (PLACEHOLDER THEORY CHOICE) ---
Lambda0 = 1.0  # single normalization [arbitrary units], set to 1.0 for a pure no-knob baseline

# Reference scales to make the ratios dimensionless
omega0 = 1.0e23      # Hz
m0_MeV = 17.0        # MeV

# Fixed exponents from theory choice (no fitting)
p1, p2, p3 = 0.0, 0.0, 0.0

def K1(omega_c, mGamma_MeV, chi):
    return Lambda0 * (omega_c/omega0)**(1/3) * (chi**p1)

def K2(omega_c, mGamma_MeV, chi):
    return Lambda0 * (mGamma_MeV/m0_MeV)**(1/2) * (chi**p2)

def K3(omega_c, mGamma_MeV, chi):
    return Lambda0 * ((omega_c/omega0)*(mGamma_MeV/m0_MeV))**(1/4) * (chi**p3)

K1_val, K2_val, K3_val = K1(omega_c, m_Gamma_MeV, chi), K2(omega_c, m_Gamma_MeV, chi), K3(omega_c, m_Gamma_MeV, chi)
K1_val, K2_val, K3_val

(1.0, 1.0, 1.0)

### Map stiffness → gauge couplings at Λ_B

We define (placeholder mapping; replace with MATH‑YM‑002 canonical form once fixed):
\[
\alpha_1(\Lambda_B) = \frac{c}{K_1^2},\quad
\alpha_2(\Lambda_B) = \frac{c}{K_2^2},\quad
\alpha_3(\Lambda_B) = \frac{c}{K_3^2},
\]
with a **single** constant `c` determined by the **single normalization choice** (e.g., fix one observable once, then never touch `c` again).

In [4]:
# Fix c by one reference normalization: we choose to match α_em(MZ) after running (deterministic closure).
# This block determines c ONCE by requiring the pipeline to reproduce alpha_em(MZ) target (no tuning elsewhere).

# 1-loop beta coefficients for SM (dg/dlnμ = (b/16π^2) g^3)
# For α^{-1}: d(α^{-1})/dlnμ = - b/(2π)
b1 = 41/6    # U(1)_Y (GUT-normalized)
b2 = -19/6   # SU(2)_L
b3 = -7      # SU(3)_c

def alpha_inv_run_1loop(alpha_inv_mu0, b, mu, mu0):
    return alpha_inv_mu0 - (b/(2*pi)) * np.log(mu/mu0)

def run_to_MZ(alpha1_LB, alpha2_LB, alpha3_LB, Lambda_B_GeV, MZ_GeV):
    a1_inv = 1.0/alpha1_LB
    a2_inv = 1.0/alpha2_LB
    a3_inv = 1.0/alpha3_LB
    a1_inv_MZ = alpha_inv_run_1loop(a1_inv, b1, MZ_GeV, Lambda_B_GeV)
    a2_inv_MZ = alpha_inv_run_1loop(a2_inv, b2, MZ_GeV, Lambda_B_GeV)
    a3_inv_MZ = alpha_inv_run_1loop(a3_inv, b3, MZ_GeV, Lambda_B_GeV)
    return 1.0/a1_inv_MZ, 1.0/a2_inv_MZ, 1.0/a3_inv_MZ

def electroweak_from_hyper_isospin(alpha1, alpha2):
    # GUT normalization: α1 = (5/3) α_Y
    alpha_Y = (3/5) * alpha1
    # α_em = α2 * α_Y / (α2 + α_Y)
    alpha_em = (alpha2*alpha_Y) / (alpha2 + alpha_Y)
    sin2_thetaW = alpha_Y / (alpha2 + alpha_Y)
    return alpha_em, sin2_thetaW

# Determine c by matching α_em(MZ) to PDG target (single normalization)
def determine_c(K1_val, K2_val, K3_val):
    # α_i(Λ_B) = c / K_i^2  => α1, α2, α3 are proportional to c
    # After running to MZ, α_em(MZ) must equal PDG target.
    # Solve for c using a simple monotonic 1D search.
    target_alpha_em = 1.0 / targets["alpha_em_inv_MZ"]
    c_lo, c_hi = 1e-8, 1e+2
    for _ in range(80):
        c_mid = (c_lo + c_hi)/2.0
        a1_LB = c_mid / (K1_val**2)
        a2_LB = c_mid / (K2_val**2)
        a3_LB = c_mid / (K3_val**2)
        a1_MZ, a2_MZ, a3_MZ = run_to_MZ(a1_LB, a2_LB, a3_LB, Lambda_B_GeV, MZ_GeV)
        aem, s2w = electroweak_from_hyper_isospin(a1_MZ, a2_MZ)
        if aem > target_alpha_em:
            c_hi = c_mid
        else:
            c_lo = c_mid
    return (c_lo + c_hi)/2.0

c_norm = determine_c(K1_val, K2_val, K3_val)
c_norm

0.021363909422278192

## Step 2 — Run RGEs down to M_Z and compare to PDG

In [5]:
# Compute gauge couplings at Λ_B
alpha1_LB = c_norm / (K1_val**2)
alpha2_LB = c_norm / (K2_val**2)
alpha3_LB = c_norm / (K3_val**2)

# Run to MZ
alpha1_MZ, alpha2_MZ, alpha3_MZ = run_to_MZ(alpha1_LB, alpha2_LB, alpha3_LB, Lambda_B_GeV, MZ_GeV)

# Derived electroweak
alpha_em_MZ, sin2_thetaW_MZ = electroweak_from_hyper_isospin(alpha1_MZ, alpha2_MZ)

res_RGE = {
    "alpha_em_inv_MZ (pred)": 1.0/alpha_em_MZ,
    "sin2_thetaW_MZ (pred)": sin2_thetaW_MZ,
    "alpha_s_MZ (pred)": alpha3_MZ,
}
res_RGE

{'alpha_em_inv_MZ (pred)': 127.95499999999996,
 'sin2_thetaW_MZ (pred)': 0.35638261283445133,
 'alpha_s_MZ (pred)': 0.0226552588387174}

In [6]:
# Compare to targets
compare = pd.DataFrame({
    "quantity": ["alpha_em_inv_MZ", "sin2_thetaW_MZ", "alpha_s_MZ"],
    "pred": [1.0/alpha_em_MZ, sin2_thetaW_MZ, alpha3_MZ],
    "target": [targets["alpha_em_inv_MZ"], targets["sin2_thetaW_MZ"], targets["alpha_s_MZ"]]
})
compare["delta"] = compare["pred"] - compare["target"]
compare

,quantity,pred,target,delta
0,alpha_em_inv_MZ,127.955000,127.95500,-4.263256e-14
1,sin2_thetaW_MZ,0.356383,0.23122,1.251626e-01
2,alpha_s_MZ,0.022655,0.11790,-9.524474e-02


## Step 3 — Predict lattice observables (MATH‑YM‑003)

We adopt the working relation (user-specified in your module text):
\[
\sigma \sim \frac{\kappa_3}{\xi_\Gamma^2},\qquad a \sim f(\alpha_3(M_Z), \Lambda_{QCD}, ...)
\]
Provide explicit definitions for \(\kappa_3\) and \(\xi_\Gamma\) in terms of \{ω_c, m_Γ, χ\} or the derived K_i, then compute:
- **string tension** \(\sigma\) (compare \(\sqrt{\sigma}\approx 440\,\mathrm{MeV}\)),
- **lattice spacing** `a` (compare to FLAG/MILC tables for chosen action).

In [7]:
# --- PLACEHOLDER MAPPINGS (replace with your canonical MATH-YM-003 definitions) ---
# Example proxies:
# kappa_3 from stiffness K3; xi_Gamma from (omega_c, m_Gamma, chi) as a coherence length scale.
# Units are illustrative; rescale once canonical mapping is finalized.

def kappa3_from_K3(K3):  # dimensionless proxy
    return K3

def xi_Gamma_from_bridge(omega_c, mGamma_MeV, chi):  # in fm (just a stand-in)
    # Coherence length ~ inverse of an energy scale; here a toy proxy
    # 1 MeV^-1 ≈ 197.3 fm; use m_Gamma to set a scale, modulated by susceptibility.
    return (197.3269804 / max(mGamma_MeV, 1e-6)) * (1.0/ max(chi, 1e-6))

kappa3 = kappa3_from_K3(K3_val)
xi_Gamma_fm = xi_Gamma_from_bridge(omega_c, m_Gamma_MeV, chi)

# String tension sigma ~ kappa3 / xi_Gamma^2
sigma_MeV2 = kappa3 / ( (197.3269804/xi_Gamma_fm)**2 )  # very rough, illustrative
sqrt_sigma_MeV = sqrt(abs(sigma_MeV2)) if sigma_MeV2>0 else float("nan")

# Lattice spacing: exemplar relation ~ xi_Gamma scaled by a factor of α_s
a_fm = xi_Gamma_fm * (alpha3_MZ / 0.12)  # scale so that α_s≈0.12 sets a≈xi_Gamma (toy)

{
    "kappa3": kappa3,
    "xi_Gamma_fm": xi_Gamma_fm,
    "sqrt_sigma_MeV (pred)": sqrt_sigma_MeV,
    "a_fm (pred)": a_fm
}

{'kappa3': 1.0,
 'xi_Gamma_fm': 13.655846394463667,
 'sqrt_sigma_MeV (pred)': 0.06920415224913495,
 'a_fm (pred)': 2.578139456069501}

> **Decision rule (closure test):** Without retuning the single normalization, do we get
> 1) \(\alpha(M_Z)\), \(\sin^2\theta_W(M_Z)\), \(\alpha_s(M_Z)\) within tolerance?  
> 2) \(\sqrt{\sigma}\approx 440\,\mathrm{MeV}\) and a within FLAG/MILC ranges?  
> If **no**, the stiffness dictionary needs revision (exponents, Bridge scale, or canonical mapping).

## Step 3 · Core Scaling Relation — Integrated & Tested

Implements the deterministic sigma-map, calibration, and lattice propagation exactly as specified in MATH–YM–003 §3 (filled).

In [8]:

import numpy as np

def sigma_core(kappa3, xi_gamma, c_sigma, delta_np=0.0):
    if xi_gamma <= 0:
        raise ValueError("xi_gamma must be positive.")
    return c_sigma * (kappa3 / (xi_gamma**2)) * (1.0 + float(delta_np))

def calibrate_c_sigma(sigma_phys, kappa3_ref, xi_gamma_ref):
    if kappa3_ref <= 0 or xi_gamma_ref <= 0:
        raise ValueError("Reference kappa3 and xi_gamma must be positive.")
    return (sigma_phys * (xi_gamma_ref**2)) / kappa3_ref

def r0_from_sigma(sigma):
    if sigma <= 0:
        raise ValueError("sigma must be positive.")
    return np.sqrt(1.65 / sigma)

def lattice_spacing_from_r0(r0, r0_over_a_lat):
    if r0_over_a_lat <= 0:
        raise ValueError("(r0/a)_lat must be positive.")
    return r0 / r0_over_a_lat

# Example test run
sigma_phys = 0.440**2  # GeV^2
kappa3_ref = 1.0 / (1.3**2)
xi_gamma_ref = 0.60

c_sigma = calibrate_c_sigma(sigma_phys, kappa3_ref, xi_gamma_ref)

kappa3_new = 1.0 / (1.2**2)
xi_gamma_new = 0.55
sigma_pred = sigma_core(kappa3_new, xi_gamma_new, c_sigma, delta_np=0.0)

r0_pred = r0_from_sigma(sigma_pred)
a_pred_gevinv = lattice_spacing_from_r0(r0_pred, r0_over_a_lat=5.3)

print(f"c_sigma = {c_sigma:.6f} GeV^0")
print(f"sigma_pred = {sigma_pred:.6f} GeV^2")
print(f"r0_pred (GeV^-1) = {r0_pred:.6f}")
print(f"a_pred (GeV^-1) = {a_pred_gevinv:.6f}")


c_sigma = 0.117786 GeV^0
sigma_pred = 0.270400 GeV^2
r0_pred (GeV^-1) = 2.470237
a_pred (GeV^-1) = 0.466082
